In [5]:
from pandas import read_csv, cut
from numpy import array, unique, log2, inf, append, where
from sklearn.model_selection import train_test_split
import random
from pprint import pprint

In [6]:
file_path = 'catalog1/cat1.csv'
df = read_csv(file_path)
df.head()
y = array(df['class'])
# df = df.rename(columns={"class": "label"})
if 'cat2.csv' in file_path:
    df.drop("Unnamed: 0.1", axis=1, inplace=True)
df.drop(["Unnamed: 0", "galex_objid", "sdss_objid", "class", "spectrometric_redshift", "pred"], axis=1, inplace=True)

In [7]:
def bucketize(dataframe, col_headers, bucket_size):
    assert len(col_headers) == len(bucket_size)
    no_of_columns = len(col_headers)
    for col in range(no_of_columns):
        labels = array([(x + 1) for x in range(bucket_size[col])])
        print(labels, col_headers[col], bucket_size[col])
        print(dataframe[col_headers[col]].head())
        temp = cut(dataframe[col_headers[col]], bucket_size[col], labels=labels)
        dataframe.drop(col_headers[col], inplace=True, axis=1)
        dataframe[col_headers[col]] = temp
    return dataframe

In [8]:
temp = bucketize(df, df.columns, [7 for x in range(len(df.columns))])
temp.head(10)

[1 2 3 4 5 6 7] u 7
0    18.286291
1    22.734962
2    21.223398
3    21.771444
4    22.767988
Name: u, dtype: float64
[1 2 3 4 5 6 7] g 7
0    16.968243
1    21.516233
2    21.143623
3    21.636995
4    22.018591
Name: g, dtype: float64
[1 2 3 4 5 6 7] r 7
0    16.334318
1    20.178501
2    21.149973
3    21.564974
4    21.658953
Name: r, dtype: float64
[1 2 3 4 5 6 7] i 7
0    16.039885
1    19.238029
2    20.998989
3    21.476393
4    21.098751
Name: i, dtype: float64
[1 2 3 4 5 6 7] z 7
0    15.857203
1    18.809675
2    21.145853
3    21.266085
4    20.734152
Name: z, dtype: float64
[1 2 3 4 5 6 7] extinction_u 7
0    0.094921
1    0.105131
2    0.085433
3    0.074157
4    0.050161
Name: extinction_u, dtype: float64
[1 2 3 4 5 6 7] extinction_g 7
0    0.073962
1    0.081917
2    0.066569
3    0.057783
4    0.039085
Name: extinction_g, dtype: float64
[1 2 3 4 5 6 7] extinction_r 7
0    0.051166
1    0.056670
2    0.046052
3    0.039974
4    0.027039
Name: extinction_r, dtype: float

,u,g,r,i,z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,...,g-z,r-i,r-z,i-z,fuv-nuv,fuv-u,fuv-g,fuv-r,fuv-i,fuv-z
0,3,2,2,2,2,3,3,3,3,3,...,4,5,4,4,3,1,1,1,1,1
1,6,7,6,5,4,3,3,3,3,3,...,7,7,6,4,5,5,5,4,3,3
2,5,6,7,6,6,3,3,3,3,3,...,3,5,3,3,4,3,3,4,4,4
3,5,7,7,7,6,2,2,2,2,2,...,3,5,4,4,4,4,5,5,5,5
4,6,7,7,7,6,1,1,1,1,1,...,5,6,5,4,5,4,4,4,3,3
5,5,6,7,6,6,2,2,2,2,2,...,3,5,4,3,5,3,4,4,4,4
6,5,6,6,6,6,2,2,2,2,2,...,3,5,4,3,5,4,4,4,4,4
7,4,5,5,5,5,1,1,1,1,1,...,3,4,4,4,4,1,1,1,2,2
8,5,6,6,6,5,1,1,1,1,1,...,4,5,4,4,5,2,3,3,3,3
9,5,7,7,7,6,1,1,1,1,1,...,3,4,3,3,4,4,5,4,4,4


In [9]:
df.head(10)

,u,g,r,i,z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z,...,g-z,r-i,r-z,i-z,fuv-nuv,fuv-u,fuv-g,fuv-r,fuv-i,fuv-z
0,3,2,2,2,2,3,3,3,3,3,...,4,5,4,4,3,1,1,1,1,1
1,6,7,6,5,4,3,3,3,3,3,...,7,7,6,4,5,5,5,4,3,3
2,5,6,7,6,6,3,3,3,3,3,...,3,5,3,3,4,3,3,4,4,4
3,5,7,7,7,6,2,2,2,2,2,...,3,5,4,4,4,4,5,5,5,5
4,6,7,7,7,6,1,1,1,1,1,...,5,6,5,4,5,4,4,4,3,3
5,5,6,7,6,6,2,2,2,2,2,...,3,5,4,3,5,3,4,4,4,4
6,5,6,6,6,6,2,2,2,2,2,...,3,5,4,3,5,4,4,4,4,4
7,4,5,5,5,5,1,1,1,1,1,...,3,4,4,4,4,1,1,1,2,2
8,5,6,6,6,5,1,1,1,1,1,...,4,5,4,4,5,2,3,3,3,3
9,5,7,7,7,6,1,1,1,1,1,...,3,4,3,3,4,4,5,4,4,4


In [10]:
X = array(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
def check_purity(y):
    if len(unique(y)) == 1:
        return True
    else:
        return False

In [12]:
def classify_data(y):
    unique_classes, counts_unique_classes = unique(y, return_counts=True)

    index = where(counts_unique_classes == max(counts_unique_classes))[0][0]
    classification = unique_classes[index]
    
    return classification

In [13]:
def get_potential_splits(X, y):
    
    potential_splits = {}
    #n_columns = len(X[0])
    for column_index in range(len(X[0])):  
        potential_splits[column_index] = []
        values = X[:, column_index]
        unique_values = unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [14]:
def split_data(X, y, split_column, split_value):
    
    no_of_columns = len(X[0]) + 1
    split_column_values = X[:, split_column]
    data_below = data_above = array([]).reshape(0, no_of_columns)
#     print("len(X) in function split_data:", len(X))
    for index in range(len(X)):
        temp = array(append(X[index], y[index])).reshape(1, no_of_columns)
        if split_column_values[index] <= split_value:
            data_below = append(data_below, temp, axis=0)
        else:
            data_above = append(data_above, temp, axis=0)
    
    return data_below, data_above

In [15]:
def calculate_entropy(data):
    
#     print(data)
    label_column = data[:, -1]
    _, counts = unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -log2(probabilities))
     
    return entropy

In [16]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [17]:
def determine_best_split(X, y, potential_splits):    
    overall_entropy = inf
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [18]:
def decision_tree_algorithm(X, y, column_headers, max_depth, counter=0, min_samples=2):
    
#     # data preparations
#     if counter == 0:
#         global COLUMN_HEADERS
#         COLUMN_HEADERS = df.columns
#         data = df.values
#     else:
#         data = df   
        
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        classification = classify_data(y)
        
        return classification

    
    # recursive part
    else:    
        counter += 1
        print(counter)

        # helper functions 
        potential_splits = get_potential_splits(X, y)
        split_column, split_value = determine_best_split(X, y, potential_splits)
        data_below, data_above = split_data(X, y, split_column, split_value)
    
        feature_name = column_headers[split_column]
        question = "column_{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below[:, :-1], data_below[:, -1], column_headers, max_depth, counter, min_samples)
        no_answer = decision_tree_algorithm(data_above[:, :-1], data_above[:, -1], column_headers, max_depth, counter, min_samples)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [19]:
import time
start_time = time.time()
tree = decision_tree_algorithm(X_train, y_train, column_headers=df.columns, max_depth=3)
end_time = time.time()
pprint(tree)
print("Time taken to construct the decision tree =", end_time - start_time)

1
2
3
2
3
{'column_22 <= 1.5': [{'column_17 <= 1.5': [{'column_13 <= 1.5': [0, 1]}, 0]},
                      {'column_13 <= 3.5': [1, {'column_29 <= 1.5': [0, 1]}]}]}
Time taken to construct the decision tree = 19.96256113052368


In [20]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")
    x = int(feature_name.split("_")[1])
    
    # ask question
    if example[x] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [21]:
def predict(X_test, tree):
    predictions = array([])
    for example in X_test:
        predictions = append(predictions, classify_example(example, tree))
    
    return predictions

In [22]:
predictions = predict(X_test, tree)

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.56      0.69        18
           1       0.96      0.99      0.98       177

    accuracy                           0.95       195
   macro avg       0.93      0.77      0.83       195
weighted avg       0.95      0.95      0.95       195



In [25]:
# Sample testing of stuff

from random import randrange
from numpy import array, log2, square
import time

N = randrange(100000, 1000000)
elements = array([randrange(1000, 10000) for x in range(N)])
print(N)

757422


In [26]:
start_time = time.time()
eles_log2 = log2(elements)
end_time = time.time()

print("Time taken to execute log2:", end_time - start_time)

Time taken to execute log2: 0.017448902130126953


In [27]:
start_time = time.time()
eles_sq = square(elements)
end_time = time.time()

print("Time taken to execute log2:", end_time - start_time)

Time taken to execute log2: 0.0021359920501708984


In [28]:
0.017448902130126953 / 0.0021359920501708984

8.168992075008372

In [5]:
def function(y):
    y += 1
    print("x in the function =", y)
    return y

x = 5
print("x in main function =", x)
temp = function(x)
print("x in main function =", x)
print("temp in main function =", temp)

x in main function = 5
x in the function = 6
x in main function = 5
temp in main function = 6
